In [1]:
%matplotlib inline

import math
import cv2

import numpy as np
import matplotlib.pyplot as plt
import ipywidgets as wd

from IPython.display import display

In [2]:
plt.rcParams["figure.figsize"] = [12, 8]

In [3]:
read = cv2.imread('/Users/yc/Downloads/samples/-421000000315_photo.jpg', cv2.IMREAD_COLOR)
original_image = cv2.cvtColor(read, cv2.COLOR_BGR2RGB)
hsv_image = cv2.cvtColor(original_image, cv2.COLOR_RGB2HSV)

def findMask_one(image, sigma_color, color):
    '''
    依照 h s v 計算距離
    '''
    image_f = image.astype(np.float)
    color_f = color.astype(np.float)
    #sigma_color_f = sigma_color.astype(np.float)
    
    dis = np.abs(image_f - color_f)
    nor = dis * np.array([1.0 / 180, 1.0 / 255, 1.0 / 255])
    return (255*(1 - np.mean(nor, axis = 2))).astype(np.uint8)

def findMask(image, sigma_color, colors):
    '''
    params
    ======
    - image: image bitmap
    - sigma_color: color space threshold
    - colors: reference colors
    
    returns
    =======
    - mask: mask of selected pixels
    '''
    mask_all = None
    for color in colors:
        
        #mask = cv2.inRange(image, color - sigma_color, color + sigma_color)
        mask = findMask_one(image, sigma_color, color)
        
        if mask_all is None:
            mask_all = mask
        else:
            mask_all = cv2.max(mask, mask_all)
        
    return mask_all

In [4]:
def pickColor(h, s, v):
    select_color = np.array([h, s, v])
    display_cube = cv2.cvtColor(select_color.reshape([1, 1, 3]).astype(np.uint8), cv2.COLOR_HSV2RGB)
    
    colors = [select_color, 
              #np.array([5, 59, 255]),
              #np.array([3, 99, 207]),
              np.array([3, 106, 94])]
    mask = findMask(hsv_image, np.array([5, 30, 30]), colors)
    #mask = cv2.blur(mask, (5, 5))
    
    ret_image = hsv_image.copy()
    #ret_image[:,:,2][mask > 0] = ((ret_image[:,:,2][mask > 0].astype(np.float) + 207) / 2).astype(np.uint8)
    #ret_image[mask > 0] = ((ret_image[mask > 0].astype(np.float) + np.array([3, 99, 207])) / 2 ).astype(np.uint8)
    #ret_image[mask > 0] = [3, 99, 207]
    ret_image = \
    mask[:,:,None].astype(np.float) / 255 * np.array([3, 99, 207], dtype=np.float) + \
    (1 - mask[:,:,None].astype(np.float) / 255) * ret_image.astype(np.float)
    
    ret_image = cv2.cvtColor(ret_image.astype(np.uint8), cv2.COLOR_HSV2RGB)
    
    number = 4
    plt.subplot(1, number, 1); plt.imshow(display_cube)
    plt.subplot(1, number, 2); plt.imshow(mask, cmap='gray')
    plt.subplot(1, number, 3); plt.imshow(ret_image)
    plt.subplot(1, number, 4); plt.imshow(original_image)
    plt.show()

h_slide_bar = wd.IntSlider(min=0, max=180, step=1, value=14)
s_slide_bar = wd.IntSlider(min=0, max=255, step=1, value=59)
v_slide_bar = wd.IntSlider(min=0, max=255, step=1, value=255)
_ = wd.interact(pickColor, h=h_slide_bar, s=s_slide_bar, v=v_slide_bar)

In [5]:
# finite diff x
# 控制項確定

# mask 演算法
# 臉部座位點 取法
# 只調整v ?